In [ ]:
TODO

- try out a different tokeniser

In [1]:
# read tinyshakespeare dataset

with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# inspect first 1000 characters
print(text[:1000])


In [7]:
# we need to tokenize the text into characters. we will have a unique token for each character.

unique_chars = set(text)
chars = sorted(list(unique_chars))
vocab_size = len(chars)

# create a mapping from characters to integers
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

print(string_to_int)
print(int_to_string)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [13]:
# playing around with the tokenization

encoded = encode("hello")
print(encoded)
print(encoded[0])
print(decode(encoded))


[46, 43, 50, 50, 53]
46
hello


In [14]:
# tokenize the text and store in a tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [15]:
# split the data into training and validation sets
ratio = 0.9 

index = int(ratio*len(data))
train_data = data[:index]
val_data = data[index:]

print(train_data.shape, val_data.shape)


torch.Size([1003854]) torch.Size([111540])


In [23]:
# CONSTANTS
BLOCK_SIZE = 8
BATCH_SIZE = 4
torch.manual_seed(42)

In [24]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    random_indices = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in random_indices])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in random_indices])
    return x, y


In [27]:
xb, yb = get_batch('train')
print(xb.shape, yb.shape)
print(xb)
print(yb)
# print decoded versions of xb and yb
for b in range(BATCH_SIZE):
    print('input:', decode(xb[b].tolist()))
    print('target:', decode(yb[b].tolist()))
    print('---')

torch.Size([4, 8]) torch.Size([4, 8])
tensor([[ 1, 57, 53,  8,  0, 21,  5, 50],
        [43, 52, 42, 57,  0, 13, 52, 42],
        [39, 58, 58, 43, 56, 12,  0,  0],
        [51, 63,  1, 57, 53, 56, 56, 53]])
tensor([[57, 53,  8,  0, 21,  5, 50, 50],
        [52, 42, 57,  0, 13, 52, 42,  1],
        [58, 58, 43, 56, 12,  0,  0, 16],
        [63,  1, 57, 53, 56, 56, 53, 61]])
input:  so.
I'l
target: so.
I'll
---
input: ends
And
target: nds
And 
---
input: atter?


target: tter?

D
---
input: my sorro
target: y sorrow
---


# Bigram

In [52]:
# bigram model
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # lookup table for each token
        
    def forward(self, idx, targets=None):
        # The embedding table maps each token to a vector of logits (scores) for next token prediction
        # Input shape (idx): [Batch_size, Time_steps] -> Output shape: [Batch_size, Time_steps, vocab_size]
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # Get logits for last position only since we only need to predict the next token
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # sample from the distribution to get the next token
            idx = torch.cat((idx, idx_next), dim=-1) # append the next token to the sequence
        return idx



In [54]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=1000)[0].tolist()))


torch.Size([32, 65])
tensor(4.6983, grad_fn=<NllLossBackward0>)

JqcOY3!ROMoDb;h:l?Zep!:?$nir:Mw UrI&WSKw'ioWoWRUjmTObPZFgwVaOMLxh
d:EJQC-SLORohtX!sytQ'nOHmH3-S!maq?b LDKv;XAa qwFA,TFgpH$YlYrGa OVDC-kj:NzEA,S!JEhvYeOQeiLygBQMiBwIc&fA3JTpC&l 
LPKi3kOMZ
d-kTrNPnpMo;,xgFOisd-FB,sPnsCUoDzTeeneaI.gZ:BmZjEqitugXr''m:wJwbZSlOHxSTD'l zAh!:QMUb.KC&UkelfC-GbYrQv 
cMiwMvhl3&.G-GHOJGh-eO?hZKyRUJuoD?,B,BjDa,bd-Tt;Q&PdVaaH$YroAVnCt'QwpwvJL:ncnHvT-'jGs$k:M?$Co.$r-kRoP-k'PniB 'QMZlnboVHpZ,Bpb uxqsJXlvT3pZbCbC?Xh$YrfYU
wyTU!dOMmTD-or!- QvnOsJXReD,szuvNcsB,q?hz 
 hoJTOjjDVMOvwJvxgcjeOJmefx ZjD:JunC,Y3nk?'H
!iyw!juFXaBA,ygAS!dAR&3M .JEHviBX.$QyqtQ?TUygXpftugXWHxaLzFa?,I&PT.I?.rKGSXHmkWSSw csLzQET gTpoHO guLwhpH-kmwoA,cOQHzc!!Q ex;QZWU?$D'3yMmeOJfwvywNQQ-cZPD$-B,U,pZwmwUGRR&wINb3ubGbgTjjt;E ay,?&lfiock$GbZAAGOTxCX.
UC-tvcm:&lnNOiR&.pkesr!IS!;cgDm:i,KehWNhagJ:Ic:wmW3k;AM?myomQ&wx;QoRn'Q-kp!!jxquoRzpZ'f:WG;QOL3e$AYryLmmezx'?APhZcYrNovrQ$HBIlu
xgbohOpS,QnsDJMcFrrn!lI3'hYrUJ!V;AqcnsUeLPxexh;VJ-pqVVGJV!d

G'fU